# Introduction to Osquery on Microsoft Windows


This is a quick introduction to [Osquery](https://osquery.io/) and a few examples of some of the queries that can be run and data returned. 

-----------------------------------------
* **Authors:** Sevickson Kwidama | Jerry Gamblin
* **Date:** 22 May 2020
* **Version:** 0.5

## Installation and Configuration

### Install needed Python libraries

We are going to attempt to install the non-standard libraries we use:
-  Pandas
-  IPython.display

***Python version >= 3.5***


In [ ]:
!pip3 install pandas --upgrade
!pip3 install IPython --upgrade

### Import the needed Python libraries

In [ ]:
import os
import json
import subprocess
import pandas as pd
from pandas import json_normalize
from IPython.display import JSON,display,HTML
import requests

### Install Osquery

These commands will download and afterward install Osquery.   
If version changes, you only need to change the `version` variable.  

Installing via Chocolatey can be found a few lines lower.  
Manual installation instructions can also be [found here](https://osquery.readthedocs.io/en/stable/installation/install-windows/). 


In [ ]:
version = "4.3.0"
url = f'https://pkg.osquery.io/windows/osquery-{version}.msi'

#First find name of file to be used for installing
if url.find('/'):
  osquery_filename = url.rsplit('/', 1)[1]

#Download file to current location
r = requests.get(url, allow_redirects=True)
open(osquery_filename, 'wb').write(r.content)

Install the Osquery MSI without interaction and also write out a log file in the same location to check if anything went wrong.

In [ ]:
!msiexec /i $osquery_filename /quiet /log "osquery-install.log"

### Install Osquery via Chocolatey (not tested!)

In [ ]:
!choco install osquery

# Using Osquery



First we need to find location of Osquery on this system, as in Windows it does not set Osquery binary in the `PATH`.  

Below only needs to be run once, it takes a few seconds to come back. If Osquery is installed on another partition than `C:\` you will need to change `path` variable.  

When you get result you can just put the value in the variable `osquery_binary`, if you do not mind the seconds it takes to find the binary you can run the code below everytime.

In [ ]:
%%time
path = "c:\\"
name = "osqueryi.exe"
for (root, dirs, files) in os.walk(path):
    if name in files:
        osquery_binary = os.path.join(root)

In [ ]:
osquery_binary

### Function definition to use for the Osquery queries

In [ ]:
def osquery_run(query):
    os.chdir(osquery_binary)
    data = subprocess.run(['osqueryi', '--json', query], stdout=subprocess.PIPE, universal_newlines=True)
    data = data.stdout
    nested = json.loads(data)
    df = pd.DataFrame(json_normalize(nested))
    df.style.set_properties(**{'text-align': 'right'})
    return df.style.hide_index()

# Basic Commands
If you want to run the commands on the CLI you only need to use the following, first change directory to `osquery_binary`:  
```
osqueryi --json "SELECT query"
```

### Display OS Version



In [ ]:
commands = "select * from os_version;"
result_query = osquery_run(commands)
result_query

### List Installed Applications

In [ ]:
commands = "select name, version from programs ORDER BY name;"
result_query = osquery_run(commands)
result_query

### List Logged In Users

In [ ]:
commands = "select * from logged_in_users;"
result_query = osquery_run(commands)
result_query

### List System Uptime

In [ ]:
commands = "SELECT * FROM uptime;"
result_query = osquery_run(commands)
result_query

### List Network Interfaces

In [ ]:
commands = "SELECT a.interface, a.address, d.mac FROM interface_addresses a JOIN interface_details d USING (interface);"
result_query = osquery_run(commands)
result_query

### List Installed Chocolatey Packages (Windows + Chocolatey)

In [ ]:
commands = "SELECT * FROM chocolatey_packages;"
result_query = osquery_run(commands)
result_query



### List System Mounted Filesystems (Windows)



In [ ]:
commands = "SELECT device_id, description, file_system, size, boot_partition FROM logical_drives;"
result_query = osquery_run(commands)
result_query
#display(df.style.hide_index().set_properties(**{'text-align': 'left'}))



### List Startup Items



In [ ]:
commands = "SELECT * FROM startup_items;"
result_query = osquery_run(commands)
result_query



### List Private Keys in ~/.ssh directory



In [ ]:
commands = "SELECT * FROM users JOIN user_ssh_keys USING (uid);"
result_query = osquery_run(commands)
result_query



### List Users



In [ ]:
commands = "SELECT username, description FROM users ORDER BY username;"
result_query = osquery_run(commands)
result_query